```python
from secret import flag
from Crypto.Util.number import getPrime, bytes_to_long

assert len(flag) == 64
m1 = bytes_to_long(flag[:32].encode())
m2 = bytes_to_long(flag[32:].encode())

p = getPrime(256)
def release(m):
    print(hex((m * pow(2, m, p)) % p)[2:].rjust(64, '0'))

print(hex(p)[2:])
release(m1)
release(m2)
release(m2-m1)
release(m2+m1)
```

In [ ]:
p = int("afe4dfec75d05b8204f949749dce9d69eaee982528f7e2c177862b4f12b635d9",16)
m1 = int("6d04f0ebde78ca72c0a65629cd6f2cc337319c05b266ed789843ea2bdf11551f",16)
m2 = int("61483d050ad72a0e6dda11e3f683fbac20ab17b4a26615ac3eb4fbaecef519bd",16)
m2Mm1 = int("13c9395628b7f90ff1675d73cc97ae24ea5c9993366364627d20f9f52b19fabb",16)
m2Pm1 = int("75e04f3f38420029fa57934de57b6fb59f9615e4be32eaa4460c57a47c2842ae",16)

In [26]:
from Crypto.Util.number import getPrime, bytes_to_long, long_to_bytes

flag = b"ABCD" * 16 

assert len(flag) == 64
flag_m1 = bytes_to_long(flag[:32])
flag_m2 = bytes_to_long(flag[32:])

assert long_to_bytes(flag_m1 + flag_m2 * 2**(8*32)) == flag

# Let's yolo with z3

In [ ]:
import z3

bv = p.bit_length()

z_p = z3.IntVal(p)
z_m1 = z3.IntVal(m1)
z_m2 = z3.IntVal(m2)
z_m2Mm1 = z3.IntVal(m2Mm1)
z_m2Pm1 = z3.IntVal(m2Pm1)

v_m1 = z3.Int("m1")
v_m2 = z3.Int("m2")

s = z3.Solver()

s.add([
    z_m1 == (v_m1 * z3.IntVal(2)**v_m1) % z_p,
    z_m2 == (v_m2 * z3.IntVal(2)**v_m1) % z_p,
    z_m2Mm1 == ((v_m1 + v_m2) * z3.IntVal(2)**(v_m1 + v_m2)) % z_p,
    z_m2Pm1 == ((v_m1 - v_m2) * z3.IntVal(2)**(v_m1 - v_m2)) % z_p   
])

Z3Exception: Z3 integer expression expected

# Okay and now yolo sage

In [18]:
from sage.all import * 

In [113]:
Fp = GF(p)
v_m1 = var('v_m1')
v_m2 = var('v_m1')

m1 = Fp(m1)
m2 = Fp(m2)
m2Mm1 = Fp(m2Mm1)
m2Pm1 = Fp(m2Pm1)

"""
s.add([
    z_m1 == (v_m1 * z3.IntVal(2)**v_m1) % z_p,
    z_m2 == (v_m2 * z3.IntVal(2)**v_m1) % z_p,
    z_m2Mm1 == ((v_m1 + v_m2) * z3.IntVal(2)**(v_m1 + v_m2)) % z_p,
    z_m2Pm1 == ((v_m1 - v_m2) * z3.IntVal(2)**(v_m1 - v_m2)) % z_p   
])
"""

# Define the equations
eq1 = m1 == (v_m1 * 2**v_m1)
eq2 = m2 == (v_m2 * 2**v_m2)
eq3 = m2Mm1 == ((v_m1 + v_m2) * 2**(v_m1 + v_m2))
eq4 = m2Pm1 == ((v_m1 - v_m2) * 2**(v_m1 - v_m2))

# Try to solve the system of equations
solutions = solve([eq1, eq2, eq3, eq4], v_m1, v_m2)

# Print the solutions
print(solutions)

[]


# Okay apparently it is by hand :()

In [120]:
from sympy import symbols,simplify

# Define symbolic variables in SymPy
v_m1, v_m2 = symbols('x y', integer=True)

# Define the equations
eq1 = (v_m1 * 2**v_m1) 
eq2 = (v_m2 * 2**v_m2) 
eq3 = ((v_m1 + v_m2) * 2**(v_m1 + v_m2))
eq4 = ((v_m1 - v_m2) * 2**(v_m1 - v_m2))


In [97]:
eq5 = simplify(eq1 * eq2)
eq5

2**(x + y)*x*y

In [98]:
eq6 = simplify(eq5 / eq3)
eq6

x*y/(x + y)

In [99]:
eq7 = simplify((eq2 * eq4))
eq7

2**x*y*(x - y)

In [100]:
eq8 = simplify(eq1 * eq7)
eq8

4**x*x*y*(x - y)

In [101]:
eq9 = simplify(eq3 * eq4)
eq9

4**x*(x - y)*(x + y)

In [102]:
eq10 = simplify((eq3 * eq4) / eq1**2)
eq10 

1 - y**2/x**2

In [110]:
eq11 = simplify(eq10 * eq6)
eq11

y*(x - y)/x

In [ ]:
# m1 = Fp(m1)
# m2 = Fp(m2)
# m2Mm1 = Fp(m2Mm1)
# m2Pm1 = Fp(m2Pm1)
v10 = (m2Mm1 * m2Pm1) / m1**2

(44937492787025696122256299424299963452933073710274843676854599456183579195484,
 79559135097231088745461412741104874367466799372248185651735720737977696400857)

In [116]:
def legendre(a,p):
    return pow(a, (p-1) // 2,p)

def tonelli(a,p):
    assert(legendre(a,p) ==1, "not a quadratic-residue")
    q = p - 1
    s = 0
    while q % 2 == 0:
        q //= 2
        s += 1

    if s == 1 :
        return pow(a, (p + 1)//4, p)

    for z in range(2, p):
        if p - 1 == legendre(z, p):
            break

    c = pow(z, q, p)
    r = pow(a, (q+1) // 2, p)
    t = pow(a, q, p)

    m = s

    t2 = 0

    while (t - 1) % p != 0:
        t2 = (t * t) % p
        for i in range(1, m):
            if (t2 -1) % p == 0:
                break
            t2 = (t2 * t2) % p
        b = pow(c, 1 << (m - i - 1), p)
        r = (r * b) % p
        c = (b * b) % p
        t = (t * c) % p
        m = i
    return r

<>:5: SyntaxWarning: assertion is always true, perhaps remove parentheses?
<>:5: SyntaxWarning: assertion is always true, perhaps remove parentheses?
/tmp/ipykernel_3553/2979477257.py:5: SyntaxWarning: assertion is always true, perhaps remove parentheses?
  assert(legendre(a,p) ==1, "not a quadratic-residue")


In [117]:
tonelli(44937492787025696122256299424299963452933073710274843676854599456183579195484,p)

ValueError: negative shift count

# BRO use GCD!! 

In [129]:
Fp = GF(p)
v_m1 = var('v_m1')
v_m2 = var('v_m1')

m1 = Fp(m1)
m2 = Fp(m2)
m2Mm1 = Fp(m2Mm1)
m2Pm1 = Fp(m2Pm1)

In [130]:
simplify(eq1 * eq2)

2**(x + y)*x*y

In [131]:
eq3

2**(x + y)*(x + y)

In [132]:
xyPow = gcd(m1*m2,m2Pm1)
xyPow

1

% Oracle definition
$
f(m) \equiv m\,2^{m}\pmod{p}
$

% Convenient shorthands
$
g_1 = 2^{M_1}\pmod{p},\qquad
g_2 = 2^{M_2}\pmod{p}.
$

% Four oracle outputs
\[
\begin{aligned}
c_1 &= M_1 g_1,\\
c_2 &= M_2 g_2,\\
c_3 &= (M_2-M_1)\,g_2 g_1^{-1},\\
c_4 &= (M_2+M_1)\,g_2 g_1.
\end{aligned}
\]

% Two relations obtained after eliminating the g_i
\[
\begin{aligned}
c_3 c_1 M_2 &= c_2 M_1 (M_2 - M_1),\\[6pt]
c_4 M_1 M_2 &= c_2 c_1 (M_2 + M_1).
\end{aligned}\tag{★}
\]

% Express M_2 in terms of M_1
$
M_2 \equiv \frac{c_2 M_1^{2}}{c_2 M_1 - c_3 c_1}\pmod{p}.
$

% Quadratic in M_1 obtained after substitution
$
(BC)\,M_1^{2} \;-\; 2BD\,M_1 \;+\; DA \equiv 0 \pmod{p},
$

$
A = c_3 c_1,\qquad
B = c_2,\qquad
C = c_4,\qquad
D = c_2 c_1.
$

% Discriminant and quadratic formula mod p
$
\Delta = b^{2} - 4ac \pmod{p},\qquad
M_1 = \frac{-b \pm \sqrt{\Delta}}{2a}\pmod{p}.
$

In [134]:
orgm1 = bytes_to_long(b'ictf{wh4t_ev3n_i5_@_r34l_w0r1d_4')
orgm2 = bytes_to_long(b'ppl1c4ti0n_9OoYVHHxYhQG6teVZXHC}')

In [136]:
(orgm1+orgm2) % p 

18967107861589613037287863598829836428835578830673904629134337754346584173784

In [140]:
gcd((100 * 18967107861589613037287863598829836428835578830673904629134337754346584173784) % p,(200 * 18967107861589613037287863598829836428835578830673904629134337754346584173784) % p)

1